In [1]:
import enum
import re
import tkinter as tk
import typing
import logging
from base64 import b64encode
from icecream import ic

from seleniumwire import webdriver

from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from time import sleep
from bs4 import BeautifulSoup
from functools import partial

from config import *
import requests
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
logging.basicConfig(level=logging.INFO)

In [3]:
class Config:
    button_check = '/html/body/app-root/div/app-candidate-application/app-candidate-application-item/div/div/div/div[3]/div/div[1]/form/div[2]/button[2]'
    send_but = '/html/body/app-root/div/app-candidate-application/app-candidate-application-item/div/div/div/div[3]/div/div[1]/div/button'
    send_but_other = '/html/body/app-root/div/app-candidate-application/app-candidate-application-item/div/div/div/div[3]/div/div[1]/form/div[2]/button[2]'
    table_body = '/html/body/app-root/div/app-candidate-applications/app-candidate-applications-list/table/tbody/tr[1]/td[1]'
    pagination = '/html/body/app-root/div/app-candidate-applications/app-candidate-applications-list/ngb-pagination[1]/ul'
    item_button = '/html/body/app-root/div/app-candidate-application/app-candidate-application-item/div/div/div/div[3]/div/div[2]/form/div/button'

config = Config()

In [4]:
class ReturnValue(enum.IntEnum):
    test = 0
    verify_or_GMS = 1

In [5]:
def send_msg(text): # функция принимает текст и отправляет его в чат в тг
    token = tg_token # тут лежит токен бота
    chat_id = "-959491185" # id чата который получает сообщение
    url_req = "https://api.telegram.org/bot" + token + "/sendMessage" + "?chat_id=" + chat_id + "&text=" + text
    return requests.get(url_req)

In [6]:
def intercept_request(request):
    ic(request)
    request.headers['Authorization'] = f"Basic {b64encode(f'{AUTH.login_admin}:{AUTH.password_admin}'.encode()).decode()}"

In [7]:
class Driver(webdriver.Chrome):
    def __init__(self, *args, seleniumwire_options=None, **kwargs):
        super().__init__(*args, seleniumwire_options=seleniumwire_options, **kwargs)
        self._current_implicitly_wait = 0
        self.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
            "source": """
                        Object.defineProperty(navigator, 'webdriver', {
                            get: () => undefined
                        })
                    """
        })

    def create_new_tab(self, url='about:blank'):
        script = f"window.open('{url}','_blank');"
        return self.execute_script(script)
        # return self.execute_cdp_cmd("Target.createTarget", {"url": url})

    def close_last_tab(self):
        ...


    @property
    def current_implicitly_wait(self):
        return self._current_implicitly_wait

    def implicitly_wait(self, seconds):
        super().implicitly_wait(seconds)
        self._current_implicitly_wait = seconds

In [8]:
driver = Driver(service=Service(ChromeDriverManager().install()))
driver.implicitly_wait(10)

INFO:WDM:====== WebDriver manager ======
INFO:WDM:Get LATEST chromedriver version for google-chrome
INFO:WDM:Get LATEST chromedriver version for google-chrome
INFO:WDM:Driver [C:\Users\1\.wdm\drivers\chromedriver\win64\134.0.6998.88\chromedriver-win32/chromedriver.exe] found in cache
INFO:seleniumwire.storage:Using default request storage
INFO:seleniumwire.backend:Created proxy listening on 127.0.0.1:59318
INFO:seleniumwire.handler:Capturing request: http://clients2.google.com/time/1/current?cup2key=8:2ue5XQ9VHL9PJLRWRm47YqV9phmWfD7V35aa75sY1Rk&cup2hreq=e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855
INFO:seleniumwire.handler:Capturing request: https://accounts.google.com/ListAccounts?gpsia=1&source=ChromiumBrowser&json=standard


In [9]:
ic("Установка интерцептора запросов")
driver.request_interceptor = intercept_request
ic("Интерцептор установлен: ", driver.request_interceptor)

ic| 'Установка интерцептора запросов'
ic| "Интерцептор установлен: ": 'Интерцептор установлен: '
    driver.request_interceptor: <function intercept_request at 0x0000018D07BC6170>


('Интерцептор установлен: ', <function __main__.intercept_request(request)>)

In [10]:
driver.get("https://office.gms.tech")

In [11]:
user = driver.find_element(By.ID, "username")
user.send_keys(AUTH.login_admin)
passw = driver.find_element(By.ID, "password")
passw.send_keys(AUTH.password_admin)
log_but = driver.find_element(By.XPATH, '/html/body/app-root/div/app-auth/div/div[3]/div/button').click()

In [12]:
def create_new_tab(url='about:blank'):
    script = f"window.open('{url}', '_blank');"
    return driver.execute_script(script)
    # return driver.execute_cdp_cmd("Target.createTarget", {"url": url})

In [68]:
# создание кнопок
root = tk.Tk()
root.title('v.1: Дорогу осилит идущий')

# Получаем разрешение экрана
screen_width = root.winfo_screenwidth()
screen_height = root.winfo_screenheight()

# Задаем размеры окна
window_width = 630
window_height = 450

# Задаем координаты окна
x = screen_width - window_width
y = 0

# Устанавливаем размеры и координаты окна
root.geometry(f"{window_width}x{window_height}+{x}+{y}")

send_msg(f'{AUTH.login_admin} запустил скрипт')

# os = 'windows'
stop_num = 100


In [14]:
def open_link(link):
    driver.create_new_tab(link)
    # driver.switch_to.new_window('tab')
    # driver.get(link)

In [15]:
def close_all_windows():
    browser_list = driver.window_handles
    for window in browser_list[1:]:
        driver.switch_to.window(window)
        driver.close()

In [16]:
def exness_vac():
    ic('Exness вакансии')
    send_msg(f'{AUTH.login_admin} открыл Exness')
    for otklik in pages_analizing():
        if otklik['profile_status'] == 'Профиль создан'and otklik['grade'] != 'C':
            if '@ Exness' in otklik['vacancy_name']:
                open_link(otklik['link'])

In [17]:
def how_many_pages():
    driver.switch_to.window(driver.window_handles[0])
    driver.get('https://office.gms.tech/candidate_applications?filter=waiting_for_review&page=1')
    try:
        num_page = int(driver.find_element(By.XPATH, config.pagination).text.split('\n')[-2].strip())
    except:
        num_page = 1

    driver.switch_to.window(driver.window_handles[0])
    ic(f"{num_page=}")
    return num_page

In [18]:
def pages_analizing():

    def soup_analize(soup):
        for i in soup.find('table', class_='table table-hover table-sm').find_all('tr')[1:]:
            otklick_dict = {'link': 'https://office.gms.tech' + i.find_all('td')[0].find('a').get('href'),
                            'vacancy_name': i.find_all('td')[2].text.strip(),
                            'grade': i.find_all('td')[4].text.strip(),
                            'profile_status': i.find_all('td')[6].text.strip(),
                            'verification_status': i.find_all('td')[7].text.strip()}
            for_analize.append(otklick_dict)

    for_analize = []
    for num in range(1, how_many_pages() + 1):
        driver.get(f'https://office.gms.tech/candidate_applications?filter=waiting_for_review&page={num}')
        elem = driver.find_element(By.XPATH, config.table_body)
        ic(elem)
        soup_analize(BeautifulSoup(driver.page_source, 'lxml'))

    return for_analize

In [19]:
def get_otkliks():
    send_msg(f'{AUTH.login_admin} открыл Только РФ')
    for otklik in pages_analizing():
        if otklik['profile_status'] == 'Профиль создан' and otklik['grade'] != 'C':
            if '@' in otklik['vacancy_name']:
                yield otklik

In [20]:
def all_vacancies(companies):
    for otklik in get_otkliks():
        if otklik['vacancy_name'].split('@')[1].strip() in companies:
            open_link(otklik['link'])

In [21]:
def specific_vacancies(vacancies):
    for otklik in get_otkliks():
        if otklik['vacancy_name'] in vacancies:
            open_link(otklik['link'])

In [22]:
def exness_check():

    def sum_exp(soup): # посчет лет опыта в профиле
        return sum([int(exp[0])*12 + int(exp[1]) for exp in [re.findall(r'\d+',exp.find('div', class_='fs-body-2 muted g-mb-8').text.split(',')[1].strip()) for exp in soup.find_all('app-experience')]])

    def otkaz_exness():
        soup = BeautifulSoup(driver.page_source, 'lxml')
        company = soup.find_all(class_='table table-hover table-sm')[1].find_all('tr')[2].find_all('td')[1].text
        check = driver.find_element(By.XPATH, config.item_button).text

        if check == 'Отказать в представлении' and company == ' #16 Exness ':
            try:
                driver.find_element(By.XPATH, config.item_button).click()
                sleep(1)
                driver.close()
            except:
                pass


    print('проверка откликов Exness')
    send_msg(f'{AUTH.login_admin} автоотказ Exness')
    browser_list = driver.window_handles
    norm_eng_level = ['C2', 'C1', 'B2']
    ne_norm_eng_level = ['B1','A2','A1', 'NA']
    for i in browser_list[1:]:
        sleep(1)
        driver.switch_to.window(i)
        soup = BeautifulSoup(driver.page_source, 'lxml')
        pars = soup.find_all('app-card')[1].find('div', class_='g-mt-20').text
        if 'Языки Английский' in pars:
            eng_level = pars.split('  : ')[1].replace(' Немецкий','').replace(' Французский','')
        else: eng_level = 'NA'

        if eng_level[:2] in norm_eng_level:
            if sum_exp(soup) < max_experience[0] and sum_exp(soup) != 0:
                otkaz_exness()
        elif eng_level[:2] in ne_norm_eng_level:
            otkaz_exness()



In [126]:

# soup = BeautifulSoup(driver.page_source, features='lxml')
# pars = soup.find_all('app-card')[1].find('div', class_='g-mt-20').text
# if 'Языки Английский' in pars:
#     eng_level = pars.split('  : ')[1].replace(' Немецкий','').replace(' Французский','').strip()
#     exp = sum([int(exp[0])*12 + int(exp[1]) for exp in [re.findall(r'\d+',exp.find('div', class_='fs-body-2 muted g-mb-8').text.split(',')[1].strip()) for exp in soup.find_all('app-experience')]])
#
#     print(eng_level[:2])

B1


In [ ]:
def refuse_close_page():
        check = driver.find_element(By.XPATH,config.item_button).text
        if check == 'Отказать в представлении':
            try:
                check.click()
                sleep(1.5)
                driver.close()
            except:
                pass

In [ ]:
def city_check(cities, countries, not_this_cities=False, not_this_countries=False): #
    ic('Закрыть вакансии')
    send_msg(f'{AUTH.login_admin} автоотказ Только РФ')
    for i in driver.window_handles[1:]:
        sleep(1)
        driver.switch_to.window(i)
        soup = BeautifulSoup(driver.page_source, 'lxml')
        location = soup.find(class_='b-profile__location').text.replace('Живёт в городе ', '').strip()
        city = location.split(',')[0]
        country = location.split(',')[1][1:]
        if (not_this_cities ^ (city in cities)) or (not_this_countries ^ (country in countries)):
            refuse_close_page()
            ic('отказал', city)

In [26]:
def open_profile_for_grade(grade):
    ic('открыть вакансии',grade)

    if grade == 1:
        grade = 'A'
    elif grade == 2:
        grade = 'B'
    elif grade == 3:
        grade = 'Не классифицирован'


    for otklik in pages_analizing():
        browser_list = driver.window_handles
        if len(browser_list) > stop_num:
            ic('too much')
            break
        if otklik['profile_status'] == 'Профиль создан':
            if otklik['grade'] == grade:
                open_link(otklik['link'])



In [27]:
def special_vac():
    ic('открыть спец вакансии')
    send_msg(f'{AUTH.login_admin} открыл спец вакансии')

    full_vacancy_name = spec_vacancies  # вставить сюда название вакансии в формате: 'UX/UI Designer @ Leroy Merlin'

    for otklik in pages_analizing():
        browser_list = driver.window_handles
        if len(browser_list) > stop_num:
            ic('too much')
            break
        if otklik['profile_status'] == 'Профиль создан' and otklik['grade'] != 'C':
            if otklik['vacancy_name'] in full_vacancy_name:
                open_link(otklik['link'])


In [28]:
def browser_list_for_auto(browser_list) -> typing.Generator[tuple[BeautifulSoup, str,str,str,str], None, None]:
    for i in browser_list[1:]:
        driver.switch_to.window(i)
        soup = BeautifulSoup(driver.page_source, 'lxml')
        grade = soup.find_all('tbody')[0].find_all('tr')[3].text.replace('Грейд', '').strip()
        comment, button_check, company = "", "", ""
        for i in soup.find_all('tbody')[1].find_all('tr'):
            if i.find_all('td')[0].text == 'Сопроводительный комментарий':
                comment = i.find_all('td')[1].text.strip()
        try:
            button_check = soup.find(class_='btn btn-success').text.strip()
        except:
            try:
                button_check = soup.find(class_='btn btn-success mr-3').text.strip()
            except:
                comment = 1
        yield soup, grade, comment, button_check, company

In [29]:
def find_company(soup):
    return [f"{i.find_all('td')[1].text}".strip() for i in soup.find_all('tbody')[1].find_all('tr') if i.find_all('td')[0].text == 'В компанию'][-1]

In [30]:
def send_but(company = ''):
    send_but = driver.find_element(By.XPATH, config.send_but_other if company != '#' else config.send_but)
    send_but.click()
    sleep(1)

In [31]:
def if_button_check(button_check) -> ReturnValue:
    if button_check == 'Тест представления':
        return ReturnValue.test
    elif button_check == 'Верифицировать' or button_check == 'Кандидат GMS готов':
        return ReturnValue.verify_or_GMS

In [32]:
def code_on_auto_a_and_auto_b(value, soup):
    ic(value)
    match value:
        case ReturnValue.test:
            driver.find_element(By.XPATH, config.button_check).click()
            sleep(1)
            driver.close()
        case ReturnValue.verify_or_GMS:
            send_but()
            company = find_company(soup)
            send_but(company)
            driver.close()
        case _:
            sleep(1)
            return


In [33]:
def auto_b_without_vac():
    browser_list = driver.window_handles
    for soup, grade, comment, button_check, company in browser_list_for_auto(browser_list):
        ic(f"{soup=}", f"{grade=}", f"{comment=}", f"{button_check=}", f"{company=}")
        code_on_auto_a_and_auto_b(if_button_check(button_check), soup)
        sleep(1.6)

In [34]:
def auto_a():
    ic('автопредставление а грейдов')
    send_msg(f'{AUTH.login_admin} автопредставления А')
    browser_list = driver.window_handles
    for soup, grade, comment, button_check, company in browser_list_for_auto(browser_list):
        if grade == 'A' and comment == '':
            code_on_auto_a_and_auto_b(if_button_check(button_check), soup)

    ic('finish')


In [35]:
def open_b_without_vac():
    ic('открыть В-грейды без откликов')
    send_msg(f'{AUTH.login_admin} открыть В-грейды без откликов')

    for otklik in pages_analizing():
        if otklik['profile_status'] == 'Профиль создан':
            if '@' not in otklik['vacancy_name'] :
                open_link(otklik['link'])



In [36]:
def gms_vac():
    ic('открыть вакансии GMS')
    send_msg(f'{AUTH.login_admin} открыть вакансии GMS')
    driver.switch_to.window(driver.window_handles[0])
    driver.get('https://office.gms.tech/candidate_applications?filter=waiting_for_review&page=1&gms=true')
    try:
        num_page = int(driver.find_element(By.XPATH,
                                           '/html/body/app-root/div/app-candidate-applications/app-candidate-applications-list/ngb-pagination[1]/ul').text.split('\n')[-2])
    except:
        num_page = 1
    driver.switch_to.window(driver.window_handles[0])
    ic(num_page)

    for_analize = []
    for num in range(1, num_page + 1):
        driver.get(f'https://office.gms.tech/candidate_applications?filter=waiting_for_review&page={num}&gms=true')
        _ = driver.find_element(By.XPATH, config.table_body)
        soup = BeautifulSoup(driver.page_source, 'lxml')
        for i in soup.find('table', class_='table table-hover table-sm').find_all('tr')[1:]:
            link = 'https://office.gms.tech' + i.find_all('td')[0].find('a').get('href')
            vacancy_name = i.find_all('td')[2].text
            grade = i.find_all('td')[4].text
            profile_status = i.find_all('td')[6].text
            verification_status = i.find_all('td')[7].text
            otklick_dict = {'link': link, 'vacancy_name': vacancy_name, 'grade': grade,
                            'profile_status': profile_status, 'verification_status': verification_status}
            for_analize.append(otklick_dict)

    for otklik in for_analize:
        if otklik['profile_status'] == 'Профиль создан':
            open_link(otklik['link'])


In [37]:
def open_all_c():
    ic('открыть все С-грейды')
    send_msg(f'{AUTH.login_admin} открыть все С-грейд')
    for otklik in pages_analizing():
        if otklik['grade'] == 'C':
            open_link(otklik['link'])


In [38]:
def next_candidate(counter, to_predst):
    driver.switch_to.window(driver.window_handles[0])
    for i in to_predst[counter[0]][1:]:
        driver.create_new_tab(f'https://office.gms.tech/{i}')
        # driver.switch_to.new_window('tab')
        # driver.get(f'https://office.gms.tech/{i}')

    counter[0] += 1

In [39]:
def cycle_predst():

    ic('Цикличное представление')
    send_msg(f'{AUTH.login_admin} цикличное представления')

    all_otklik = []
    for num in range(1, how_many_pages() + 1):
        driver.get(f'https://office.gms.tech/candidate_applications?filter=waiting_for_review&page={num}')
        elem = driver.find_element(By.XPATH, config.table_body)
        ic(elem.text)
        sleep(3)
        soup = BeautifulSoup(driver.page_source, 'lxml')
        for i in soup.find(class_='table table-hover table-sm').find_all('tr')[1:]:
            if i.find_all('td')[6].text == 'Профиль создан':
                mail = i.find_all('td')[3].text.split(' ')[1]
                link = i.find_all('td')[0].find('a').get('href')
                all_otklik.append([mail, link])

    all_mails = set()
    for i in all_otklik:
        all_mails.add(i[0])
    to_predst = []
    for i in all_mails:
        candidate = [i]
        for n in all_otklik:
            if n[0] == i: candidate.append(n[1])
        to_predst.append(candidate)
    to_predst.sort(key=len, reverse=True)

    counter = [0]



    panel_window = tk.Toplevel(root)
    panel_window.title('v.1')

    panel_window.title('v.1: Дорогу осилит идущий')

    # Получаем разрешение экрана
    screen_width = panel_window.winfo_screenwidth()

    # Задаем размеры окна
    p_window_width = 150
    p_window_height = 100

    # Задаем координаты окна
    x = screen_width - p_window_width
    y = 0

    # Устанавливаем размеры и координаты окна
    panel_window.geometry(f"{p_window_width}x{p_window_height}+{x}+{y}")

    # Создаем кнопки и добавляем их на панель
    button1 = tk.Button(panel_window, text="следующий", command=partial(next_candidate, counter, to_predst))
    button1.pack(padx=10, pady=10)

    button2 = tk.Button(panel_window, text="Закрыть окно", command=panel_window.destroy)
    button2.pack(padx=10, pady=10)


In [40]:
def how_many_vac():
    ic('Подсчёт вакансий')
    send_msg(f'{AUTH.login_admin} подсчет вакансий')

    number = 0
    for num in range(1, how_many_pages() + 1):
        driver.get(f'https://office.gms.tech/candidate_applications?filter=waiting_for_review&page={num}')
        sleep(3)
        ic(num)
        soup = BeautifulSoup(driver.page_source, 'lxml')
        for i in soup.find(class_='table table-hover table-sm').find_all('tr')[1:]:
            if i.find_all('td')[6].text == 'Профиль создан': number += 1
    ic('Сейчас надо представить:',number,'вакансий')


In [69]:
class NextNum:
    __counts = {0: 0}

    def __call__(self, column):
        self.__counts[column] = self.__counts.get(column, 1) + 1
        return {'row': self.__counts[column], "column": column}

In [70]:


eng_level_text = """Для Exness
(автоотказ для английского ниже В2"""
how_many_vac_text = """Сколько
вакансий
надо представить"""

def change_exp():
    exp = experience.get()
    try:
        max_experience[0] = round(float(exp) * 12)
        print(max_experience)
        experience_ERR['text'] = ''
    except ValueError:
        experience_ERR['text'] = 'Введите число'



experience_text = tk.Label(root, text='Установить проходной стаж')
experience = tk.Entry(root)
experience.insert(0, max_experience[0] / 12)
experience_OK = tk.Button(root, text='ОК', command=change_exp)
experience_ERR = tk.Label(root, text='', fg='red', font=('Arial', '20', 'bold'))

close_all = tk.Button(root, text="закрыть окна", command=close_all_windows)
exness = tk.Button(root, text="Exness", command=exness_vac)
eng_level = tk.Button(root, text=eng_level_text, command=exness_check,fg="red")
rus_all = tk.Button(root, text="РФ все вакансии (открыть)", command=partial(all_vacancies, russia.companies))
rus_spec = tk.Button(root, text="РФ только по должностям (открыть)", command=partial(specific_vacancies, russia.vacancies))
rus_auto_check = tk.Button(root, text="Только РФ (автоотказ)", command = lambda: city_check(russia.cities, ['Russia','Россия']),fg="red")
no_rus_vac_all = tk.Button(root, text="Вне РФ все вакансии (открыть)", command=partial(all_vacancies, other_countries.companies))
no_rus_vac_spec = tk.Button(root, text="Вне РФ только по должностям (открыть)", command=partial(all_vacancies, other_countries.vacancies))
no_rus_auto_check = tk.Button(root, text="Вне РФ (автоотказ)", command=lambda: city_check(other_countries.cities, ['Russia','Россия'], False, True), fg="red")
grade_a = tk.Button(root, text="А (открыть)", command=partial(open_profile_for_grade, 1))
grade_b = tk.Button(root, text="В (открыть)", command=partial(open_profile_for_grade, 2))
grade_without = tk.Button(root, text="Без грейда (открыть)", command=partial(open_profile_for_grade, 3))
special = tk.Button(root, text="Специальные вакансии (открыть)", command=special_vac)
auto_a_grade = tk.Button(root, text="Автопредставления А грейдов", command=auto_a,fg="red")
auto_b_outvac = tk.Button(root, text="Верифицировать В (!Без вакансий!)", command=auto_b_without_vac,fg="red")
auto_gms_outvac = tk.Button(root, text="Автопредставление GMS", command=auto_b_without_vac,fg="red")
open_b_without = tk.Button(root, text="Открыть В грейды без вакансий", command=open_b_without_vac)
gms = tk.Button(root, text="GMS", command=gms_vac)
open_c_grade = tk.Button(root, text="Открыть все С", command=open_all_c)
cycle_button = tk.Button(root, text="Цикличное  представление", command=cycle_predst)
how_many_vac = tk.Button(root, text=how_many_vac_text, command=how_many_vac)
#test_f = tk.Button(root, text="test", command=test,fg="red")
quit = tk.Button(root, text="Quit", command=root.destroy)

# experience['state'] = tk.DISABLED

auto_row = NextNum()

close_all.grid(**auto_row(1))

exness.grid(**auto_row(0))
rus_all.grid(**auto_row(0))
rus_spec.grid(**auto_row(0))
no_rus_vac_all.grid(**auto_row(0))
no_rus_vac_spec.grid(**auto_row(0))
special.grid(**auto_row(0))
grade_a.grid(**auto_row(0))
open_b_without.grid(**auto_row(0))
gms.grid(**auto_row(0))
grade_b.grid(**auto_row(0))
grade_without.grid(**auto_row(0))
open_c_grade.grid(**auto_row(0))
cycle_button.grid(**auto_row(0))

eng_level.grid(**auto_row(2))
rus_auto_check.grid(**auto_row(2))
no_rus_auto_check.grid(**auto_row(2))
auto_a_grade.grid(**auto_row(2))
auto_b_outvac.grid(**auto_row(2))
auto_gms_outvac.grid(**auto_row(2))

how_many_vac.grid(**auto_row(1))

quit.grid(**auto_row(1))
experience_text.grid(**auto_row(1))
experience.grid(**auto_row(1))
experience_OK.grid(**auto_row(1))
experience_ERR.grid(**auto_row(1))


root.mainloop()

ic| 'Exness вакансии'
ic| f"{num_page=}": 'num_page=9'
ic| elem: <selenium.webdriver.remote.webelement.WebElement (session="7e711c86eb85c175bad61c9ab1940e5c", element="f.F90C188DA99E98F2045B7982CF65120F.d.69FB7D2794E3528A17719EE33936A34D.e.45")>
ic| elem: <selenium.webdriver.remote.webelement.WebElement (session="7e711c86eb85c175bad61c9ab1940e5c", element="f.F90C188DA99E98F2045B7982CF65120F.d.B340360136ECDAB59BA9056823E5CEA5.e.60")>
ic| elem: <selenium.webdriver.remote.webelement.WebElement (session="7e711c86eb85c175bad61c9ab1940e5c", element="f.F90C188DA99E98F2045B7982CF65120F.d.EAAACD4300549056897D86A84F1BA66A.e.72")>
ic| elem: <selenium.webdriver.remote.webelement.WebElement (session="7e711c86eb85c175bad61c9ab1940e5c", element="f.F90C188DA99E98F2045B7982CF65120F.d.BD5589C3C364B851C7C48AE9E117A928.e.80")>
ic| elem: <selenium.webdriver.remote.webelement.WebElement (session="7e711c86eb85c175bad61c9ab1940e5c", element="f.F90C188DA99E98F2045B7982CF65120F.d.46BF7F2B6ECC49ECB6A19260F10DC8C